# Gotland

This notebook runs [GOTM](https://gotm.net/) using the [Gotland deep](https://gotm.net/cases/gotland_deep/) test case. 

Run the case with four turbulence closure schemes:

- GLS-C01A: The generic length scale ([Umlauf and Burchard, 2003](https://doi.org/10.1357/002224003322005087)) model in the $k$-$\epsilon$ formulation with the weak-equilibrium stability function by [Canuto et al., 2001](https://doi.org/10.1175/1520-0485(2001)031%3C1413:OTPIOP%3E2.0.CO;2) (C01A).

- Three variants of KPP via [CVMix](http://cvmix.github.io):
    - KPP-CVMix ([Large et al., 1994](https://doi.org/10.1029/94RG01872), [Griffies et al., 2015](https://github.com/CVMix/CVMix-description/raw/master/cvmix.pdf))
    - KPPLT-VR12 ([Li et al., 2016](https://doi.org/10.1016%2Fj.ocemod.2015.07.020))
    - KPPLT-LF17 ([Li and Fox-Kemper, 2017](https://doi.org/10.1175%2FJPO-D-17-0085.1))
    
Stokes drift profiles are computed in a similar way as in [Li et al., 2017](https://doi.org/10.1016/j.ocemod.2017.03.016), with constraints of wave variables from a wave hindcase simulation.

In [1]:
import sys
import copy
import numpy as np
import matplotlib.pyplot as plt
# add the path of gotmtool
sys.path.append("../gotmtool")
from gotmtool import *

## Create a model
Create a model with environment file `../../.gotm_env.yaml`, which is created by `gotm_env_init.py`. 

In [2]:
m = Model(name='Gotland_wave', environ='../gotmtool/.gotm_env.yaml')

Take a look at what are defined in the environment file.

In [3]:
for key in m.environ:
    print('{:>15s}: {}'.format(key, m.environ[key]) )

   gotmdir_code: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/code
   gotmdir_data: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/data
  gotmdir_build: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/build
    gotmdir_exe: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/exe
    gotmdir_run: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/run
 gotmdir_figure: /Users/qingli/work/A2020_CVMix_in_GOTM/gotm/figure
   gotmdir_tool: /Users/qingli/work/A2020_CVMix_in_GOTM/gotmtool


## Build the model

In [4]:
%%time
m.build()

CPU times: user 3.11 ms, sys: 8.5 ms, total: 11.6 ms
Wall time: 78.7 ms


## Configuration
Initialize the GOTM configuration

In [5]:
cfg = m.init_config()

Generating default configuration at '/Users/qingli/work/A2020_CVMix_in_GOTM/gotm/run/Gotland_wave/gotm.yaml'...
Done!


Update the configuration

In [6]:
# setup
title = 'Gotland Basin'
nlev = 250
cfg['title'] = title
cfg['location']['name'] = 'Station 271'
cfg['location']['latitude'] = 57.3
cfg['location']['longitude'] = 20.0
cfg['location']['depth'] = 250.0
cfg['time']['start'] = '1997-01-01 00:00:00'
cfg['time']['stop']  = '2002-01-01 00:00:00'
cfg['time']['dt']    = 600.0
cfg['grid']['nlev']  = nlev
cfg['grid']['ddu']   = 0.0

# output
cfg['output']['gotm_out']['title'] = title
cfg['output']['gotm_out']['k1_stop'] = nlev+1
cfg['output']['gotm_out']['k_stop'] = nlev
cfg['output']['gotm_out']['time_unit'] = 'day'
cfg['output']['gotm_out']['time_step'] = 1

# forcing
datadir = os.getcwd()+'/data'
cfg['temperature']['method'] = 2
cfg['temperature']['file'] = datadir+'/t_prof.dat'
cfg['salinity']['method'] = 2
cfg['salinity']['file'] = datadir+'/s_prof.dat'
cfg['salinity']['relax']['tau'] = 4320000.0
cfg['salinity']['relax']['tau_s'] = 4320000.0
cfg['salinity']['relax']['tau_b'] = 4320000.0
cfg['surface']['fluxes']['method'] = 1
cfg['surface']['meteo']['u10']['method'] = 2
cfg['surface']['meteo']['u10']['file'] = datadir+'/meteo.dat'
cfg['surface']['meteo']['u10']['column'] = 1
cfg['surface']['meteo']['u10']['scale_factor'] = 1.0
cfg['surface']['meteo']['v10']['method'] = 2
cfg['surface']['meteo']['v10']['file'] = datadir+'/meteo.dat'
cfg['surface']['meteo']['v10']['column'] = 2
cfg['surface']['meteo']['v10']['scale_factor'] = 1.0
cfg['surface']['meteo']['airp']['method'] = 2
cfg['surface']['meteo']['airp']['file'] = datadir+'/meteo.dat'
cfg['surface']['meteo']['airp']['column'] = 3
cfg['surface']['meteo']['airp']['scale_factor'] = 1.0
cfg['surface']['meteo']['airt']['method'] = 2
cfg['surface']['meteo']['airt']['file'] = datadir+'/meteo.dat'
cfg['surface']['meteo']['airt']['column'] = 4
cfg['surface']['meteo']['airt']['scale_factor'] = 1.0
cfg['surface']['meteo']['hum']['method'] = 2
cfg['surface']['meteo']['hum']['file'] = datadir+'/meteo.dat'
cfg['surface']['meteo']['hum']['column'] = 5
cfg['surface']['meteo']['hum']['scale_factor'] = 1.0
cfg['surface']['meteo']['hum']['type'] = 1
cfg['surface']['meteo']['cloud']['method'] = 2
cfg['surface']['meteo']['cloud']['file'] = datadir+'/meteo.dat'
cfg['surface']['meteo']['cloud']['column'] = 6
cfg['surface']['meteo']['cloud']['scale_factor'] = 1.0
cfg['surface']['meteo']['swr']['method'] = 3
cfg['surface']['meteo']['precip']['method'] = 2
cfg['surface']['meteo']['precip']['file'] = datadir+'/precip.dat'

# Stokes drift
cfg['stokes_drift']['us']['method'] = 2
cfg['stokes_drift']['us']['file'] = datadir+'/us_prof.dat'
cfg['stokes_drift']['us']['column'] = 1
cfg['stokes_drift']['vs']['method'] = 2
cfg['stokes_drift']['vs']['file'] = datadir+'/us_prof.dat'
cfg['stokes_drift']['vs']['column'] = 2


# water type
cfg['light_extinction']['method'] = 7
cfg['light_extinction']['A']['method'] = 0
cfg['light_extinction']['A']['constant_value'] = 0.4
cfg['light_extinction']['g1']['method'] = 0
cfg['light_extinction']['g1']['constant_value'] = 3
cfg['light_extinction']['g2']['method'] = 0
cfg['light_extinction']['g2']['constant_value'] = 15.7

# EOS 
cfg['eq_state']['mode'] = 2
cfg['eq_state']['method'] = 2

# buoyancy
cfg['buoyancy']['NN_ini'] = 1.0e-4

## Run the model

Set the configurations and labels for each run

In [7]:
cfgs = []
labels = []

In [8]:
cfg['turbulence']['turb_method'] = 3
cfg['turbulence']['tke_method'] = 2
cfg['turbulence']['len_scale_method'] = 10
cfg['turbulence']['scnd']['method'] =  2
cfg['turbulence']['scnd']['scnd_coeff'] =  5
cfg['turbulence']['turb_param']['length_lim'] = 'false'
cfg['turbulence']['turb_param']['compute_c3'] = 'true'
cfg['turbulence']['turb_param']['Ri_st'] = 0.25
cfg['turbulence']['generic']['gen_m'] = 1.5 
cfg['turbulence']['generic']['gen_n'] = -1.0
cfg['turbulence']['generic']['gen_p'] = 3.0 
cfg['turbulence']['generic']['cpsi1'] = 1.44
cfg['turbulence']['generic']['cpsi2'] = 1.92
cfg['turbulence']['generic']['cpsi3minus'] = -0.63
cfg['turbulence']['generic']['cpsi3plus'] = 1.0 
cfg['turbulence']['generic']['sig_kpsi'] = 1.0 
cfg['turbulence']['generic']['sig_psi'] = 1.3
cfgs.append(copy.deepcopy(cfg))
labels.append('GLS-C01A')

In [9]:
cfg['turbulence']['turb_method'] = 100
cfgs.append(copy.deepcopy(cfg))
labels.append('KPP-CVMix')

In [10]:
cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 1
cfgs.append(copy.deepcopy(cfg))
labels.append('KPPLT-VR12')

In [11]:
cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 2
cfgs.append(copy.deepcopy(cfg))
labels.append('KPPLT-LF17')

Run the three cases in parallel with 2 processes 

In [12]:
%%time
sims = m.run_batch(configs=cfgs, labels=labels, nproc=2)

CPU times: user 63.6 ms, sys: 24.7 ms, total: 88.3 ms
Wall time: 1min 58s
